In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare
from models.Encoders import NRMS_Encoder, FIM_Encoder, NPA_Encoder, Pipeline_Encoder, MHA_Encoder
from models.SFI import SFI_gating

In [2]:
name='sfi'
hparams = {
    'scale':'demo',
    'mode':'train',
    'batch_size':10,
    'title_size':20,
    'his_size':50,
    'npratio':4,
    'dropout_p':0.2,
    'embedding_dim':300,
    'value_dim':16,
    'query_dim':200,
    'head_num':16,
    'filter_num':150,
    'level':1,
    'epochs':5,
    'metrics':'auc,mean_mrr,ndcg@5,ndcg@10',
    'device':'cuda:1',
    'attrs': ['title'],
    'k':10,
    'save_step':[0],
    'train_embedding':True,
    'news_id':True,
    'validate':False
}
torch.cuda.set_device(1)

In [3]:
vocab, loaders = prepare(hparams)

[2021-03-15 14:34:49,862] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'value_dim': 16, 'query_dim': 200, 'head_num': 16, 'filter_num': 150, 'level': 1, 'epochs': 5, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'k': 10, 'save_step': [0], 'train_embedding': True, 'news_id': True, 'validate': False}
[2021-03-15 14:34:49,864] INFO (root) preparing dataset...
[2021-03-15 14:34:52,075] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'nrms',hparams['select']])
nrms_encoder = NRMS_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, nrms_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'fim',hparams['select']])

fim_encoder = FIM_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, fim_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'npa',hparams['select']])

hparams['user_dim'] = 200
hparams['query_dim'] = 200
hparams['filter_num'] = 400

npa_encoder = NPA_Encoder(hparams, vocab, len(loaders[0].dataset.uid2index))
sfi = SFI_gating(hparams, npa_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'pipeline',hparams['select']])

hparams['pipeline'] = 'sfi-fim'
# hparams['scale'] = 'large'

ppl_encoder = Pipeline_Encoder(hparams)
sfi = SFI_gating(hparams, ppl_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [ ]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'mha',hparams['select']])

mha_encoder = MHA_Encoder(hparams, vocab)
sfi = SFI_gating(hparams, mha_encoder).to(hparams['device'])

In [ ]:
sfi = train(sfi, hparams, loaders, interval=10)

In [3]:
hparams['select'] = 'gating'
hparams['name'] = '-'.join([name,'bert',hparams['select']])

hparams['bert'] = 'bert-base-uncased'
_, loaders = prepare(hparams)

[2021-03-15 14:52:54,295] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'his_size': 50, 'npratio': 4, 'dropout_p': 0.2, 'embedding_dim': 300, 'value_dim': 16, 'query_dim': 200, 'head_num': 16, 'filter_num': 150, 'level': 1, 'epochs': 5, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:1', 'attrs': ['title'], 'k': 10, 'save_step': [0], 'train_embedding': True, 'news_id': True, 'validate': False, 'select': 'gating', 'name': 'sfi-bert-gating', 'bert': 'bert-base-uncased'}
[2021-03-15 14:52:54,296] INFO (root) preparing dataset...


In [4]:
bert_encoder = Bert_Encoder(hparams)
sfi = SFI_gating(hparams, bert_encoder).to(hparams['device'])

sfi = train(sfi, hparams, loaders, interval=10)

## The rest is for development

In [8]:
record = next(iter(loaders[0]))